# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import copy
import pickle

import torch
from torch_geometric.data import Data

import wandb

import sbmlcore
import sbmlsim
import gumpy

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
full_dataset = torch.load('full_orig_dataset.pth')

### Set Up Params and Sweep Config

In [12]:
seed = 42
np.random.seed(seed)
random.seed(seed)

n_samples = 10000

cutoff_distance = 6.3  
# gcn params
num_node_features = 12
batch_size = 256
hidden_channels = 64
learning_rate = 0.001
wd = 5e-5
epochs = 400

ews = 'none'

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [13]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    # 'optimizer': {
    #     'values': ['adam', 'sgd']
    #     },
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

sweep_config['parameters'] = parameters_dict

# parameters_dict.update({
#     'epochs': {
#         'value': 400}
#     })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'log_uniform_values',
        'min': 1e-6,
        'max': 1e-1
      },
    # 'batch_size': {
    #     # integers between 32 and 256
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform_values',
    #     'q': 8,
    #     'min': 32,
    #     'max': 256,
    #   }
    })



In [14]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'hidden_channels': {'values': [32, 64, 128, 256]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.1,
                                  'min': 1e-06}}}


### Define Training Loop

In [ ]:
model, train_acc, test_acc, train_loss, test_loss= run_model.pnca_simpleGCN(
    # sequences=sequences_dict,
    sequences=None,
    self_loops = False,
    cutoff_distance = cutoff_distance,
    # edge_weight_func = '1-(dist/cutoff)',
    edge_weight_func= 'none',
    batch_size = batch_size,
    num_node_features = num_node_features,
    hidden_channels = hidden_channels,
    learning_rate = learning_rate,
    wd = wd,
    epochs = epochs,
    dataset = full_dataset,
    normalise_ews=True,
    # wandb_params={
    #     'use_wandb' : False,
    #     'wandb_project' : 'PncA Carter Dataset',
    #     'wandb_name' : f'Initial run',
    #     'n_samples' : n_samples
    # }
)

torch.save(model, f'saved_models/carter_ds_aug/es_adamw_n{n_samples}_seed{seed}_{datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")}')

In [15]:
# sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Create sweep with ID: oewyn196
Sweep URL: https://wandb.ai/dylan-home/pytorch-sweeps-demo/sweeps/oewyn196


In [ ]:
model, train_acc, test_acc, train_loss, test_loss= run_model.pnca_simpleGCN(
    # sequences=sequences_dict,
    sequences=None,
    self_loops = False,
    cutoff_distance = cutoff_distance,
    # edge_weight_func = '1-(dist/cutoff)',
    edge_weight_func= 'none',
    batch_size = batch_size,
    num_node_features = num_node_features,
    hidden_channels = hidden_channels,
    learning_rate = learning_rate,
    wd = wd,
    epochs = epochs,
    dataset = full_dataset,
    normalise_ews=True,
    # wandb_params={
    #     'use_wandb' : False,
    #     'wandb_project' : 'PncA Carter Dataset',
    #     'wandb_name' : f'Initial run',
    #     'n_samples' : n_samples
    # }
)

torch.save(model, f'saved_models/carter_ds_aug/es_adamw_n{n_samples}_seed{seed}_{datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")}')